# SpreadSheet Munging Strategies in Python - Pivot Tables - Complex Unpivoting

## __Pivot Tables - Complex Unpivoting__

*updated : September 26, 2024*

This is part of a series of blog posts about extracting data from spreadsheets using Python.  It is based on the [book](https://nacnudus.github.io/spreadsheet-munging-strategies/index.html) written by [Duncan Garmonsway](https://twitter.com/nacnudus?lang=en), which was written primarily for R users. Links to the other posts are on the sidebar.

The key takeaway is this - you understand your data layout; use the tools to achieve your end goal. [xlsx_cells](https://pyjanitor-devs.github.io/pyjanitor/api/io/#janitor.io.xlsx_cells) offers a way to get the cells in a spreadsheet into individual rows, with some metadata. The final outcome however relies on your understanding of the data layout and its proper application.

We've dealt with pivot tables in one of the [previous posts](Pivot-Tables---Simple-Unpivoting.ipynb). Here, we take the complexity up a notch. Let's dive in to the various scenarios.

(case-1)=
### **Case 1 : Centre-aligned headers**
![pivot-centre-aligned.png](Images/pivot-centre-aligned.png)

In this case, the headers are not aligned completely with the subjects or names. If the data is read into Pandas, columns B and C are set as the index of the dataframe, and a forward/backward fill applied, "Humanities" could be wrongly assigned to "Music" or "Performance" to " Literature" ("Music" should be paired with "Performance", while "Humanities" should be paired with "Literature"). Same goes for the header columns - if rows 2 and 3 are read in as header columns, and a forward/backward fill applied,  "Female" may be wrongly assigned to "Lenny", while "Male" could be wrongly assigned to "Olivia". 

![highlight-borders.png](Images/centred_aligned_headers_borders.jpg)

The solution is to get the coordinates for the horizontal and vertical borders, and use that to correctly pair the header rows and header columns. We'll use [xlsx_cells](https://pyjanitor-devs.github.io/pyjanitor/api/io/#janitor.io.xlsx_cells) function from [pyjanitor](https://pyjanitor-devs.github.io/pyjanitor/) to get the coordinates - under the hood, it uses [openpyxl](https://openpyxl.readthedocs.io/en/stable/index.html):

In [140]:
# pip install pyjanitor
import pandas as pd
import janitor as jn
import numpy as np
import sys

In [141]:
print("pandas version: ", pd.__version__)
print("janitor version: ", jn.__version__)
print("python version: ", sys.version)
print("numpy version: ", np.__version__)

pandas version:  2.2.2
janitor version:  0.29.1
python version:  3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:51:49) [Clang 16.0.6 ]
numpy version:  2.0.2


In [142]:
excel_file = pd.ExcelFile("Data_files/worked-examples.xlsx")

In [143]:
frame = jn.xlsx_cells(
    excel_file,
    sheetnames="pivot-centre-aligned",
    border=True,
    include_blank_cells=False,
)
frame = frame.astype({"row": np.int8, "column": np.int8})
frame.head()

,value,internal_value,coordinate,row,column,data_type,is_date,number_format,border
0,Female,Female,E2,2,5,s,False,General,"{'left': {'style': None, 'color': None}, 'righ..."
1,Male,Male,I2,2,9,s,False,General,"{'left': {'style': None, 'color': None}, 'righ..."
2,Leah,Leah,D3,3,4,s,False,General,"{'left': {'style': 'thin', 'color': None}, 'ri..."
3,Matilda,Matilda,E3,3,5,s,False,General,"{'left': {'style': None, 'color': None}, 'righ..."
4,Olivia,Olivia,F3,3,6,s,False,General,"{'left': {'style': None, 'color': None}, 'righ..."
...,...,...,...,...,...,...,...,...,...
70,8,8,F11,11,6,n,False,General,"{'left': {'style': None, 'color': None}, 'righ..."
71,6,6,G11,11,7,n,False,General,"{'left': {'style': 'thin', 'color': None}, 'ri..."
72,1,1,H11,11,8,n,False,General,"{'left': {'style': None, 'color': None}, 'righ..."
73,12,12,I11,11,9,n,False,General,"{'left': {'style': None, 'color': None}, 'righ..."


**Observations**:
1. The first row is the gender; the next row are the names.
2. The male and female students are separated by a vertical border. We'll use the border position to accurately reshape the data.
3. The first column are the fields; the next column are the subjects.
4. Just like the gender/names; a border separates the fields, horizontally.
5. The scores are all integers.

In [144]:
# get the fields
# the very first column
min_col = frame.column.min()
fields = frame.loc[frame.column == min_col, ["value"]]
fields = fields.rename(columns={"value": "field"})
# the horizontal borders to properly pair field and subject
# a good option is the bottom border
booleans = frame.border.str.get("bottom").str.get("style")
rows = frame.loc[booleans.notna(), "row"].array
# align with the first cell per window
# the first subject is one below the border
# hence the +1
rows = np.unique(rows)[: len(fields)] + 1
fields["row"] = rows
subjects = frame.loc[frame.column == (min_col + 1), ["value", "row"]]
subjects = subjects.rename(columns={"value": "subject"})
# merge with fields:
fields_and_subjects = subjects.merge(fields, on="row", how="left").ffill()
fields_and_subjects

,subject,row,field
0,Classics,4,Humanities
1,History,5,Humanities
2,Literature,6,Humanities
3,Philosophy,7,Humanities
4,Languages,8,Humanities
5,Music,9,Performance
6,Dance,10,Performance
7,Drama,11,Performance


In [145]:
# gender is the very first row
min_row = frame.row.min()
gender = frame.loc[frame.row == min_row, ["value"]]
gender = gender.rename(columns={"value": "gender"})
# the vertical borders we are interested in
# are the ones paired with the numbers
# a good option is the left border
border_booleans = frame.border.str.get("left").str.get("style").notna()
number_booleans = frame.data_type.eq("n")
booleans = border_booleans & number_booleans
columns = frame.loc[booleans, "column"].array
# align with the first cell per window
columns = np.unique(columns)[: len(gender)]
gender["column"] = columns
# nemes are the very next row after gender
names = frame.loc[frame.row == (min_row + 1), ["value", "column"]]
names = names.rename(columns={"value": "name"})
# merge with gender
names_and_gender = names.merge(gender, on="column", how="left").ffill()
names

,name,column
2,Leah,4
3,Matilda,5
4,Olivia,6
5,Lenny,7
6,Max,8
7,Nicholas,9
8,Paul,10


In [146]:
# grab the scores
data = frame.loc[number_booleans, ["value", "row", "column"]]
data = data.rename(columns={"value": "score"})
# merge with fields_and subjects
# merge with names_and_gender
outcome = (
    data.merge(fields_and_subjects, on="row", how="left")
    .merge(names_and_gender, on="column", how="left")
    .loc[:, ["name", "gender", "field", "subject", "score"]]
)

outcome

,name,gender,field,subject,score
0,Leah,Female,Humanities,Classics,3
1,Matilda,Female,Humanities,Classics,1
2,Olivia,Female,Humanities,Classics,2
3,Lenny,Male,Humanities,Classics,4
4,Max,Male,Humanities,Classics,3
5,Nicholas,Male,Humanities,Classics,3
6,Paul,Male,Humanities,Classics,0
7,Leah,Female,Humanities,History,8
8,Matilda,Female,Humanities,History,3
9,Olivia,Female,Humanities,History,4


### **Case 2: Repeated rows/columns of headers within the table**
![pivot-repeated-headers.png](Images/pivot-repeated-headers.png)

**Observations** : <br>
The row header (Term1, Term2, Term3) is repeated in four locations; we only need one.<br>
The index columns are clearly delineated; the pairing of subjects and names is assured. 

For this, we skip [xlsx_cells](https://pyjanitor-devs.github.io/pyjanitor/api/io/#janitor.io.xlsx_cells), and take advantage of pandas' [MultiIndexes](https://pandas.pydata.org/docs/user_guide/advanced.html):

In [147]:
(
    excel_file.parse(
        sheet_name="pivot-repeated-headers", header=[0, 1], index_col=[0, 1, 2]
    )
    .droplevel(axis=0, level=0)
    .droplevel(axis=1, level=0)
    .transform(pd.to_numeric, errors="coerce")
    .dropna()
    .astype(np.int8)
    .stack(future_stack=True)
    .rename("scores")
    .rename_axis(index=["subject", "name", "term"])
    .reset_index()
)

,subject,name,term,scores
0,Classics,Matilda,Term 1,1
1,Classics,Matilda,Term 2,7
2,Classics,Matilda,Term 3,4
3,Classics,Nicholas,Term 1,2
4,Classics,Nicholas,Term 2,9
5,Classics,Nicholas,Term 3,2
6,Classics,Olivia,Term 1,8
7,Classics,Olivia,Term 2,4
8,Classics,Olivia,Term 3,9
9,Classics,Paul,Term 1,9


### **Case 3 : Headers amongst the data**
![pivot-header-within-data.png](Images/pivot-header-within-data.png)

In this scenario, we have the subjects as a row header, mixed with the data(`classics`, `history`, `music`, `drama`). Note that the Term1,Term2,Term3 row is repeated.

Let's use [xlsx_cells](https://pyjanitor-devs.github.io/pyjanitor/api/io/#janitor.io.xlsx_cells) to get the font data (specifically, the cells with bold font) and with that location, correctly align the data:

In [148]:
frame = jn.xlsx_cells(
    excel_file,
    sheetnames="pivot-header-within-data",
    font=True,
    border=True,
    include_blank_cells=False,
)
frame = frame.astype({"row": np.int8, "column": np.int8})
frame.head()

,value,internal_value,coordinate,row,column,data_type,is_date,number_format,font,border
0,Classics,Classics,C2,2,3,s,False,General,"{'name': 'Calibri', 'family': 2.0, 'sz': 11.0,...","{'left': {'style': 'thin', 'color': None}, 'ri..."
1,Term 1,Term 1,C3,3,3,s,False,General,"{'name': 'Calibri', 'family': 2.0, 'sz': 11.0,...","{'left': {'style': 'thin', 'color': None}, 'ri..."
2,Term 2,Term 2,D3,3,4,s,False,General,"{'name': 'Calibri', 'family': 2.0, 'sz': 11.0,...","{'left': {'style': None, 'color': None}, 'righ..."
3,Term 3,Term 3,E3,3,5,s,False,General,"{'name': 'Calibri', 'family': 2.0, 'sz': 11.0,...","{'left': {'style': None, 'color': None}, 'righ..."
4,Matilda,Matilda,B4,4,2,s,False,General,"{'name': 'Calibri', 'family': 2.0, 'sz': 11.0,...","{'left': {'style': 'thin', 'color': None}, 'ri..."
...,...,...,...,...,...,...,...,...,...,...
75,3,3,E24,24,5,n,False,General,"{'name': 'Calibri', 'family': 2.0, 'sz': 11.0,...","{'left': {'style': None, 'color': None}, 'righ..."
76,Paul,Paul,B25,25,2,s,False,General,"{'name': 'Calibri', 'family': 2.0, 'sz': 11.0,...","{'left': {'style': 'thin', 'color': None}, 'ri..."
77,5,5,C25,25,3,n,False,General,"{'name': 'Calibri', 'family': 2.0, 'sz': 11.0,...","{'left': {'style': 'thin', 'color': None}, 'ri..."
78,7,7,D25,25,4,n,False,General,"{'name': 'Calibri', 'family': 2.0, 'sz': 11.0,...","{'left': {'style': None, 'color': None}, 'righ..."


Observations:
- The subjects are in bold font
- The subjects are one row above the Terms
- The names are aligned with the scores row wise
- The term is one row above the scores; this implies the shortest distance between the scores and the subjects is 2

In [149]:
subjects = frame.loc[frame.font.str.get("b"), ["value", "row"]]
subjects = subjects.rename(columns={"value": "subject"})
terms = frame.loc[frame.row.isin(subjects.row + 1), ["value", "row", "column"]]
terms = terms.rename(columns={"value": "term"})
subjects["row"] += 1  # align with the terms
# merge term and subjects:
terms_subjects = subjects.merge(terms, on="row")
terms_subjects["row"] += 1  # align with the immediate next cell
# names have a left border
border_booleans = frame.border.str.get("left").str.get("style").notna()
string_booleans = frame.data_type.eq("s")
booleans = border_booleans & string_booleans
names = frame.loc[booleans, ["value", "row"]]
names = names.rename(columns={"value": "name"})
number_booleans = frame.data_type.eq("n")
numbers = frame.loc[number_booleans, ["value", "row", "column"]]
# sort columns so that merge with term and subject will be in the proper order
numbers = numbers.rename(columns={"value": "score"}).sort_values(["column", "row"])
# merge into one DataFrame
# numbers.merge(names, on='row', how='left')
outcome = (
    numbers.merge(terms_subjects, on=["column", "row"], how="left")
    .ffill()
    .merge(names, on="row")
    .loc[:, ["name", "subject", "term", "score"]]
)
outcome

,name,subject,term,score
0,Matilda,Classics,Term 1,7
1,Nicholas,Classics,Term 1,9
2,Olivia,Classics,Term 1,5
3,Paul,Classics,Term 1,1
4,Matilda,History,Term 1,0
5,Nicholas,History,Term 1,4
6,Olivia,History,Term 1,1
7,Paul,History,Term 1,3
8,Matilda,Music,Term 1,6
9,Nicholas,Music,Term 1,5


Another route, without [xlsx_cells](https://pyjanitor-devs.github.io/pyjanitor/api/io/#janitor.io.xlsx_cells):

In [150]:
(
    excel_file.parse(sheet_name="pivot-header-within-data")
    .dropna(axis=1, how="all")
    .set_axis(["name", "Term 1", "Term 2", "Term3"], axis="columns")
    .assign(
        subject=lambda f: f["Term 1"]
        .where(~f["Term 1"].str.startswith("Term", na=False) & f.name.isna())
        .ffill()
    )
    .dropna()
    .melt(id_vars=["name", "subject"], var_name="term", value_name="score")
)

,name,subject,term,score
0,Matilda,Classics,Term 1,7
1,Nicholas,Classics,Term 1,9
2,Olivia,Classics,Term 1,5
3,Paul,Classics,Term 1,1
4,Matilda,History,Term 1,0
5,Nicholas,History,Term 1,4
6,Olivia,History,Term 1,1
7,Paul,History,Term 1,3
8,Matilda,Music,Term 1,6
9,Nicholas,Music,Term 1,5


## Comments
<script src="https://utteranc.es/client.js"
        repo="samukweku/data-wrangling-blog"
        issue-term="title"
        theme="github-light"
        crossorigin="anonymous"
        async>
</script>